# Lab 7: Cleanup

This lab removes all AWS resources created during the workshop to avoid ongoing charges.

## Step 1: Import Cleanup Utilities

In [ ]:
import os
os.environ["CLAUDE_CODE_USE_BEDROCK"] = "1"
os.environ.pop("CLAUDECODE", None)

from utils.aws_helpers import (
    get_ssm_parameter,
    delete_ssm_parameter,
    agentcore_memory_cleanup,
    gateway_target_cleanup,
    runtime_resource_cleanup,
    delete_agentcore_runtime_execution_role,
    cleanup_cognito_resources,
    delete_customer_support_secret,
    delete_observability_resources,
    local_file_cleanup,
    policy_engine_cleanup,
)

print("Cleanup utilities loaded.")

## Step 2: Clean Up Memory Resources

In [ ]:
print("=" * 60)
print("Cleaning up AgentCore Memory...")
print("=" * 60)
try:
    memory_id = get_ssm_parameter("/app/customersupport/agentcore/memory_id")
    agentcore_memory_cleanup(memory_id)
    delete_ssm_parameter("/app/customersupport/agentcore/memory_id")
    print("Memory cleanup complete.")
except Exception as e:
    print(f"Memory cleanup skipped: {e}")

## Step 3: Clean Up Runtime Resources

In [ ]:
print("=" * 60)
print("Cleaning up AgentCore Runtime...")
print("=" * 60)
try:
    runtime_arn = get_ssm_parameter("/app/customersupport/agentcore/runtime_arn")
    runtime_resource_cleanup(runtime_arn)
    delete_ssm_parameter("/app/customersupport/agentcore/runtime_arn")
    print("Runtime cleanup complete.")
except Exception as e:
    print(f"Runtime cleanup skipped: {e}")

## Step 4: Clean Up Gateway and Policy Engine

In [ ]:
print("=" * 60)
print("Cleaning up AgentCore Gateway...")
print("=" * 60)
try:
    gateway_id = get_ssm_parameter("/app/customersupport/agentcore/gateway_id")
    gateway_target_cleanup(gateway_id)
    delete_ssm_parameter("/app/customersupport/agentcore/gateway_id")
    print("Gateway cleanup complete.")
except Exception as e:
    print(f"Gateway cleanup skipped: {e}")

print("\nCleaning up Policy Engine...")
try:
    policy_engine_cleanup()
    print("Policy engine cleanup complete.")
except Exception as e:
    print(f"Policy engine cleanup skipped: {e}")

## Step 5: Clean Up Security Resources

In [ ]:
print("=" * 60)
print("Cleaning up security resources...")
print("=" * 60)

# Delete execution role
print("\n1. Deleting execution role...")
delete_agentcore_runtime_execution_role()

# Delete SSM parameters
print("\n2. Deleting SSM parameters...")
ssm_params = [
    "/app/customersupport/agentcore/client_id",
    "/app/customersupport/agentcore/pool_id",
    "/app/customersupport/agentcore/cognito_discovery_url",
    "/app/customersupport/agentcore/client_secret",
    "/app/customersupport/agentcore/runtime_execution_role_arn",
]
for param in ssm_params:
    delete_ssm_parameter(param)
    print(f"  Deleted: {param}")

# Clean up Cognito
print("\n3. Cleaning up Cognito...")
try:
    pool_id = get_ssm_parameter("/app/customersupport/agentcore/pool_id")
    cleanup_cognito_resources(pool_id)
except Exception as e:
    print(f"  Cognito cleanup skipped: {e}")

# Delete secrets
print("\n4. Deleting secrets...")
delete_customer_support_secret()

print("\nSecurity cleanup complete.")

## Step 6: Clean Up Local Files and Observability

In [ ]:
print("=" * 60)
print("Cleaning up local files...")
print("=" * 60)
local_file_cleanup()

print("\nCleaning up observability resources...")
delete_observability_resources()

## Step 7: Clean Up CloudFormation Stacks

Run the cleanup script to remove the prerequisite infrastructure.

In [ ]:
!cd scripts && bash cleanup.sh

## Cleanup Complete

All resources have been cleaned up:

| Resource | Status |
|----------|--------|
| AgentCore Memory | Deleted |
| AgentCore Runtime + ECR | Deleted |
| AgentCore Gateway + Targets | Deleted |
| Policy Engine + Policies | Deleted |
| IAM Role + Policy | Deleted |
| Cognito Pool + Users | Deleted |
| Secrets Manager | Deleted |
| SSM Parameters | Deleted |
| CloudWatch Logs | Deleted |
| CloudFormation Stacks | Deleted |
| Local Files | Cleaned |

Thank you for completing the Customer Support Agent with Claude Agent SDK workshop!